## Importing necessary python libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

### Reading the data

In [ ]:
data= pd.read_csv("emp_rating.csv")

In [ ]:
data.head()

## We can see that we have only numerical variables and emp_id is just a unique identification number of the employee. Lets set it as index

In [ ]:
data.set_index('Emp Id',inplace = True)

### descriptive stats of the dataset

In [ ]:
data.describe()

## Visualize the spread of the variables using a boxplot

In [ ]:
data.boxplot(figsize = (12,8), grid = True)
plt.suptitle('BOXPLOTS TO CHECK THE SPREAD OF VARIABLES', size = 20)

## Question-1: Do you find any difference in the scale of the variables? Is there an exception? Write your observations from the boxplots.

NO I didnt find any difference in the scale of the variable

Yes I see one exception that in the sports column range is only from 0 to 50 but remaining all the column are ranges between 50 

to 250. so it will provided differences in the dataset.

From the boxplot it will see that the column Nature,Theatre and Shopping  have consist of some kind of outliers and other column 

does not have any kind of outliers.Again it will see that sports column have less range compared to other column.

## Lets check the co-relation matrix to understand the relationships

In [ ]:
plt.figure(figsize = (10,8) )
ax = sns.heatmap(data.corr(),annot = True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

## Question-2: What patterns do you observe here? What insights can you draw from the exploratory data analysis so far?

The correlation coefficient is bound between -1 and 1 and tells you the linear relationship between these two variables. A 
Cofficient closes to 1 it is shows strong positive correlation as it is goes to negative it will show strong negative 
correlation

From the correlation plot it will see that picnic and sports are highly positive correlated because it is +0.8

From the correlation plot it will see that Religious and Shopping are highly positive correlated because it is +0.9

from this insight are the shopping behaviour are strongly belongs to the religious what religious it will be belongs

from the insight are the most of the going to picnic are playing the cricket in the picnic spot

From the correlation plot it will see that Nature and Religious are highly negative correlated because it is -0.15

it will see that there is no pattern among the nature and religious

In [ ]:
from sklearn.preprocessing import MinMaxScaler

data_copy = data[:]
scaler = MinMaxScaler()
data_copy[data_copy.columns] = scaler.fit_transform(data_copy)

## Question-3: What did we do here? Why did we do it? 

Firstly I have impored MinMaxScaler from sklearn.preprocessing 

Then I have stored the dataset on variable data_copy of including all the columns and rows

Then you have done scaling to normalize the dataset that scales the feature so that they have coverted from μ=0 and σ=1

Then you have done scaler.fit_transform that Transform features by scaling each feature to a given range. This estimator scales  
translates each feature individually such that it is in the given range on the training set, e.g. between 0 and 1.

### `Hopkins Statistic`

In [ ]:
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
import numpy as np
from math import isnan
 
def my_hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
    m = int(0.1 * n) 
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    for j in range(0, m):
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
 
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0
 
    return H

In [ ]:
my_hopkins(data_copy)

## Question-4: According to Hopkins Statistic is there cluster tendency in the data?

The Hopkins statistic, is a statistic which gives a value which indicates the cluster tendency, in other words: how well the data can be clustered.

From this dataset Hopkins satistics is 0.815399 so it has a high tendency to form a cluster.

## Model building - KMeans

In [ ]:
from sklearn.cluster import KMeans

### Choosing K by comparing Cost against each K¶

In [ ]:
import pylab as pl
number_of_clusters = range(1,5)
kmeans = [KMeans(n_clusters=i,max_iter=1000,random_state=42) for i in number_of_clusters]
score = [-1*kmeans[i].fit(data_copy).score(data_copy) for i in range(len(kmeans))]
pl.plot((number_of_clusters),score)
pl.xlabel('Number of Clusters')
pl.ylabel('Score')
pl.title('Elbow Curve')
pl.show()

## According to the Elbow Curve the optimum No of Clusters is 2. But, your management plans 3 different types of incentives to give to the customers. Hence we will be running K-Means with number of clusters as 3

## Question-5: Write the code for creating the KMeans clusters with number of clusters = 3.

### Hint: Use the function KMeans(n_clusters= no. of clusters ,max_iter=1000,random_state=42)

In [ ]:
# Write your code here
km=KMeans(n_clusters=3,max_iter=1000,random_state=42,verbose=2)
fitClusters = km.fit_predict(data_copy)

### Combining the predicted clusters with the original DF.¶

In [ ]:
copy_2 = data_copy.reset_index()
clustersDf = pd.DataFrame(fitClusters)
data['k_mean_pred'] = fitClusters
clustersDf.columns = ['k_mean_pred']
combinedDf = pd.concat([copy_2, clustersDf], axis = 1)
combinedDf.set_index('Emp Id',inplace = True)
combinedDf.head()

## Analysing the segments based on the clusters created by K-means clustering Model

In [ ]:
temp = ['K means clustering'] 
a = 0
for x in  ['k_mean_pred']:
    plt.figure(figsize = (15,8))
    c = 1
    for variables in  data.columns[:-2]:
        plt.subplot(2,3,c)
        sns.boxplot(x= x, y= variables , data= data)
        c +=1

        plt.subplots_adjust(wspace=0.50)
    plt.suptitle('Different Segments of Employees - ' + temp[a])
    a +=1

## Now that the segments have been created, we need to profile each segment for reporting

## Question-6: What are the major differences between Employee Segment 0 and Employee Segment 1?

The Difference between Employee Segment 0 and Employee Segment 1 as follows:

Employee Segment 0 is not well distributed it is skewed to one whisker for all the columns such as sport , religion , nature , theatre and shopping and there will be outlier for columns sports and theatre

Employee Segment 1 it is not well distributed throughout the column such as sport , religion , nature , theatre and shopping  and there will be outlier for columns Nature

## Question-7: Which of the employee segments does not show much interest in any kind of leisure activity or entertainment ?

Employee Segment 0 is does not show much interest in any kind of leisure activity or entertainment because it has been whiskes at a one point compared to the other segment.